# 分词部分

In [1]:
import os 
import re

import jieba
import json
import pandas as pd

In [2]:
### ONLY FOR TEST
def test_segmentation(poem, user_dict=None):   
    for s in poem:
        seg_list = jieba.cut(s, cut_all=False)  
        print('/'.join(list(seg_list)))
        
def run_test_1(user_dict=None):
    poem = ['秦川雄帝宅，函谷壯皇居。', '綺殿千尋起，離宮百雉餘。', '連甍遙接漢，飛觀迥凌虛。', '雲日隱層闕，風煙出綺疎。']
    print('Poem #1')
    test_segmentation(poem, user_dict)
    
    poem = ['遠山媚平楚，宿雨漲清溪。', '沿泝任舟檝，歡言無町畦。', '夜深忽梦少年事，梦啼妆泪红阑干']
    print('Poem #2')
    test_segmentation(poem, user_dict)
    
    poem = ['剑外忽传收蓟北，初闻涕泪满衣裳。',
            '却看妻子愁何在，漫卷诗书喜欲狂。',
            '白日放歌须纵酒，青春作伴好还乡。',
            '即从巴峡穿巫峡，便下襄阳向洛阳。']
    print('Poem #3')
    test_segmentation(poem, user_dict)
    
    poem = ['明月几时有，把酒问青天。', '不知天上宫阙，今夕是何年。', '我欲乘风归去，又恐琼楼玉宇，高处不胜寒。', '起舞弄清影，何似在人间。',
              '转朱阁，低绮户，照无眠。', '不应有恨，何事长向别时圆。', '人有悲欢离合，月有阴晴圆缺，此事古难全。', '但愿人长久，千里共婵娟。']
    print('Poem #4')
    test_segmentation(poem, user_dict)

In [3]:
LYRIC_DIR = './ci/'
POEM_DIR = './json/'

## 唐诗

先测试一下jieba分词的原始分词功能，以下诗句明显分错：
+ 秦川/雄帝/宅/
+ 沿/泝/任舟/檝/，/歡言/無/町/畦/
+ /月/有/阴晴圆/缺/

In [4]:
run_test_1()

Building prefix dict from the default dictionary ...


Poem #1


Dumping model to file cache C:\Users\JUNHONG\AppData\Local\Temp\jieba.cache
Loading model cost 0.615 seconds.
Prefix dict has been built successfully.


秦川/雄帝/宅/，/函谷壯/皇居/。
綺/殿/千尋起/，/離宮/百雉/餘/。
連/甍/遙/接漢/，/飛觀/迥凌虛/。
雲日/隱層/闕/，/風煙出/綺/疎/。
Poem #2
遠山媚/平楚/，/宿雨漲/清溪/。
沿/泝/任舟/檝/，/歡言/無/町/畦/。
夜深/忽梦/少年/事/，/梦啼妆泪/红/阑干
Poem #3
剑外/忽/传收/蓟北/，/初闻/涕泪/满/衣裳/。
却/看/妻子/愁/何在/，/漫卷/诗书/喜欲狂/。
白日/放歌/须/纵酒/，/青春/作伴/好/还乡/。
即/从/巴峡/穿/巫峡/，/便/下/襄阳/向/洛阳/。
Poem #4
明月/几时/有/，/把酒/问青天/。
不知/天上/宫阙/，/今夕/是/何年/。
我/欲/乘风/归去/，/又/恐/琼楼玉宇/，/高处不胜/寒/。
起舞/弄清/影/，/何似/在/人间/。
转朱阁/，/低绮户/，/照无眠/。
不/应有/恨/，/何事/长向/别时圆/。
人有/悲欢离合/，/月/有/阴晴圆/缺/，/此事/古难全/。
但愿人长久/，/千里/共/婵娟/。


接下来我的想法是，先根据五字句的前两个字和七字句的前四个字构建词典，然后以此作为分词词典对整句进行分词。

我用了全唐诗和全宋诗作为数据集来构建词典。经测试，比只使用全唐诗更加精确。

In [5]:
poems = []
file_list = os.listdir(POEM_DIR)
for file in file_list:
    if file.startswith('poet.'):
        poems.append(pd.read_json(f'{POEM_DIR}{file}'))
poem_df = pd.concat(poems, ignore_index=True)

In [6]:
def cut_lines(poem):
    lines = []
    pattern = r',|\.|/|;|\'|`|\[|\]|<|>|\?|:|"|\{|\}|\~|!|@|#|\$|%|\^|&|\(|\)|-|=|\_|\+|，|。|、|；|‘|’|【|】|·|！| |…|（|）'
    for pairs in poem:
        lines.extend(list(filter(lambda x: len(x) > 0, re.split(pattern, pairs))))
    return lines

In [7]:
poem_df['lines'] = poem_df['paragraphs'].apply(cut_lines)

In [8]:
def first_segment(lines):
    words = []
    for line in lines:
        if len(line) == 5: 
            words.append(line[0:2])
        if len(line) == 7: 
            words.append(line[0:2])
            words.append(line[2:4])
    return words

In [9]:
poem_df['dic'] = poem_df['lines'].apply(first_segment)

In [10]:
word_freq = dict()

dictionary = [item for sublist in poem_df['dic'] for item in sublist]
for i in dictionary:
    word_freq[i] = word_freq.get(i, 0) + 1

word_freq = {key:val for key, val in word_freq.items() if val >= 3}

with open('./poem_dict.txt', 'w', encoding='utf-8') as f:
    for k, v in word_freq.items():
        f.write(f'{k} {v}\n')

jieba.set_dictionary('./poem_dict.txt')

测试目前词典的分词效果。可见之前使用原始词典分错的地方都能分对，但是也添加了几处错误：
+ /梦啼妆泪/红/阑/干
+ /初闻涕/泪满/衣裳/
+ 我欲/乘风/归/去/

但是目前我们只关心唐诗的五字句和七字句的后三个字的分词，所以可以认为效果明显变好，毕竟“雄/帝宅”，“任/舟檝”，“無/町畦”等都能分对。

In [11]:
run_test_1()

Building prefix dict from C:\Users\JUNHONG\history-digital\poem_dict.txt ...


Poem #1


Dumping model to file cache C:\Users\JUNHONG\AppData\Local\Temp\jieba.u8c6200af4b343395bea15d660881757b.cache
Loading model cost 0.239 seconds.
Prefix dict has been built successfully.


秦川/雄/帝宅/，/函谷/壯/皇居/。
綺/殿/千尋/起/，/離宮/百雉/餘/。
連甍/遙接漢/，/飛觀/迥/凌虛/。
雲日/隱層/闕/，/風煙/出/綺疎/。
Poem #2
遠山/媚/平楚/，/宿雨/漲/清溪/。
沿泝/任/舟檝/，/歡言/無/町畦/。
夜深/忽梦/少年/事/，/梦啼妆泪/红/阑/干
Poem #3
剑外/忽/传收/蓟北/，/初闻涕/泪满/衣裳/。
却看/妻子/愁/何在/，/漫卷/诗书/喜欲狂/。
白日/放歌/须/纵酒/，/青春/作伴/好/还/乡/。
即/从/巴峡/穿/巫峡/，/便下/襄阳/向/洛阳/。
Poem #4
明月/几时/有/，/把酒/问/青天/。
不知/天上/宫阙/，/今夕/是何/年/。
我欲/乘风/归/去/，/又恐/琼楼/玉宇/，/高处/不胜寒/。
起舞/弄/清影/，/何似/在人/间/。
转朱阁/，/低绮户/，/照无眠/。
不应/有恨/，/何事/长向别/时/圆/。
人有/悲欢/离合/，/月有/阴晴/圆缺/，/此事/古难全/。
但愿/人长/久/，/千里/共婵娟/。


之后根据词典对后三个字再次进行分词，并据此扩充词典：

In [12]:
def addition_segment(phrase):
    result = jieba.lcut(phrase, cut_all=False)     
    if len(result) != 2:
        return phrase
    if len(result[0]) == 2:
        return result[0]
    return result[1]

def second_segment(lines):
    words = []
    for line in lines:
        if len(line) == 5:
            words.append(line[0:2])
            words.append(addition_segment(line[2:]))
        if len(line) == 7:
            words.append(line[0:2])
            words.append(line[2:4])
            words.append(addition_segment(line[4:]))
    return words

In [13]:
poem_df['dic_total'] = poem_df['lines'].apply(second_segment)

In [14]:
word_freq = dict()
dictionary = [item for sublist in poem_df['dic_total'] for item in sublist]
for i in dictionary:
    word_freq[i] = word_freq.get(i, 0) + 1

word_freq = {key:val for key, val in word_freq.items() if val >= 3}

with open('./whole_poem_dict.txt', 'w', encoding='utf-8') as f:
    for k, v in word_freq.items():
        f.write(f'{k} {v}\n')

jieba.set_dictionary('./whole_poem_dict.txt')

第二次的分词效果如下，可见效果再次变好，我觉得和上次的分词效果相当，虽然修正了“初闻/涕泪/满/衣裳”，但是“却看/妻子/愁何/在”又分错了。

In [15]:
run_test_1()

Building prefix dict from C:\Users\JUNHONG\history-digital\whole_poem_dict.txt ...


Poem #1


Dumping model to file cache C:\Users\JUNHONG\AppData\Local\Temp\jieba.u62ee47001558428dfaffbfcc81625e42.cache
Loading model cost 0.276 seconds.
Prefix dict has been built successfully.


秦川/雄/帝宅/，/函谷/壯/皇居/。
綺/殿/千尋/起/，/離宮/百雉/餘/。
連甍/遙接漢/，/飛觀/迥/凌虛/。
雲日/隱層/闕/，/風煙/出/綺疎/。
Poem #2
遠山/媚/平楚/，/宿雨/漲/清溪/。
沿泝/任/舟檝/，/歡言/無/町畦/。
夜深/忽梦/少年/事/，/梦啼妆泪/红/阑/干
Poem #3
剑外/忽/传收/蓟北/，/初闻/涕泪/满/衣裳/。
却看/妻子/愁何/在/，/漫卷/诗书/喜欲狂/。
白日/放歌/须/纵酒/，/青春/作伴/好/还/乡/。
即/从/巴峡/穿/巫峡/，/便下/襄阳/向/洛阳/。
Poem #4
明月/几时/有/，/把酒/问/青天/。
不知/天上/宫阙/，/今夕/是何/年/。
我欲/乘风/归/去/，/又恐/琼楼/玉宇/，/高处/不胜寒/。
起舞/弄/清影/，/何似/在人/间/。
转朱阁/，/低绮户/，/照无眠/。
不应/有恨/，/何事/长向别/时/圆/。
人有/悲欢/离合/，/月有/阴晴/圆缺/，/此事/古难全/。
但愿/人长/久/，/千里/共婵娟/。


总结，jieba分词很有问题，词典中单词的频率并不能有效改善分词结果。比如在自定义的词典中，明明没有“愁何”，而且“何在”出现了很多次，但jieba仍然分成了“愁何/在”。

jieba的这个问题已经讨论了好几年了：
https://github.com/fxsjy/jieba/issues/14

## 宋词

我本想人工划分意向，但有些名句出现得太例外：
+ 六字句通常划分方式各异，最常见的为222式，但有的名句与众不同：七八个星天外，两三点雨山前。
+ “满江红”此句与别句的划分方式都不同：“驾长车，踏破贺兰山缺”，加之“贺兰山”是地名，故最好不要人工划分。

所以只能重回之前用的套路，先提取五字句中的前两个字和七字句中的前四字扩充词典，再根据扩充后的词典分词。

In [16]:
lyrics = []
file_list = os.listdir(LYRIC_DIR)
for file in file_list:
    if file.startswith('ci.song'):
        lyrics.append(pd.read_json(f'{LYRIC_DIR}{file}'))
lyric_df = pd.concat(lyrics, ignore_index=True)
lyric_df

,author,paragraphs,rhythmic,tags,prologue
0,和岘,"[气和玉烛，睿化著鸿明。, 缇管一阳生。, 郊禋盛礼燔柴毕，旋轸凤凰城。, 森罗仪卫振华缨。...",导引,NaN,NaN
1,和岘,"[严夜警，铜莲漏迟迟。, 清禁肃，森陛戟，羽卫俨皇闱。, 角声励，钲鼓攸宜。, 金管成雅奏，...",六州,NaN,NaN
2,和岘,"[承宝运，驯致隆平。, 鸿庆被寰瀛。, 时清俗阜，治定功成。, 遐迩咏由庚。, 严郊祀，文物...",十二时・忆少年,NaN,NaN
3,王禹,"[雨恨云愁，江南依旧称佳丽。, 水村渔市。, 一缕孤烟细。, 天际征鸿，遥认行如缀。, 平生...",点绛唇,NaN,NaN
4,苏易简,"[神仙神仙瑶池宴。, 片片。, 碧桃零落春风晚。, 翠云开处，隐隐金舆挽。, 玉麟背冷清风远。]",越江吟,NaN,NaN
...,...,...,...,...,...
21048,范成大,"[催下珠帘护绮丛。, 花枝红里烛枝红。, 烛光花影夜葱茏。, 锦地绣天香雾里，珠星璧月彩云中...",浣溪沙,NaN,NaN
21049,范成大,"[送尽残春更出游。, 风前踪迹似沙鸥。, 浅斟低唱小淹留。, 月见西楼清夜醉，雨添南浦绿波愁...",浣溪沙,NaN,NaN
21050,范成大,"[歙浦钱塘一水通。, 闲云如幕碧重重。, 吴山应在碧云东。, 无力海棠风淡荡，半眠官柳日葱茏...",浣溪沙,NaN,NaN
21051,范成大,"[宝髻双双出绮丛。, 妆光梅影各春风。, 收灯时候却相逢。, 鱼子笺中此宛转，龙香拨上语玲珑...",浣溪沙,NaN,NaN


词牌名概览。出现次数最多的15个词牌名占了宋词的三分之一：

In [17]:
rhythm_df = lyric_df.groupby(['rhythmic']).count()
rhythm_df.sort_values(by='paragraphs', ascending=False).head(15)

,author,paragraphs,tags,prologue
rhythmic,,,,
浣溪沙,776,776,16,1
水调歌头,743,743,45,0
鹧鸪天,657,657,19,0
菩萨蛮,598,598,9,0
满江红,549,549,30,0
西江月,490,490,7,2
临江仙,482,482,13,0
念奴娇,432,432,28,0
减字木兰花,425,425,4,0


In [18]:
lyric_df['lines'] = lyric_df['paragraphs'].apply(cut_lines)

In [19]:
lyric_df['dic'] = lyric_df['lines'].apply(first_segment)
lyric_df['dic']

0        [睿化, 缇管, 郊禋, 盛礼, 旋轸, 森罗, 仪卫, 载路, 皇图, 大业, 垂象, 九...
1                         [铜莲, 羽卫, 金管, 属景, 盛德, 三秀, 合尧, 蹈咏]
2        [鸿庆, 遐迩, 星拱, 布羽, 明德, 神降, 万乘, 肃銮, 玉帛, 皇泽, 普率, 睿...
3                                     [江南, 依旧, 一缕, 遥认, 谁会]
4                             [神仙, 神仙, 碧桃, 零落, 隐隐, 玉麟, 背冷]
                               ...                        
21048     [催下, 珠帘, 花枝, 红里, 烛光, 花影, 锦地, 绣天, 珠星, 璧月, 人间, 别有]
21049     [送尽, 残春, 风前, 踪迹, 浅斟, 低唱, 月见, 西楼, 雨添, 南浦, 有人, 无计]
21050     [歙浦, 钱塘, 闲云, 如幕, 吴山, 应在, 无力, 海棠, 半眠, 官柳, 眼前, 春色]
21051     [宝髻, 双双, 妆光, 梅影, 收灯, 时候, 鱼子, 笺中, 龙香, 拨上, 明朝, 车马]
21052     [红锦, 障泥, 解鞍, 呼渡, 马骄, 不肯, 茅店, 竹篱, 绛裙, 青袂, 临平, 风物]
Name: dic, Length: 21053, dtype: object

In [20]:
word_freq = dict()
dictionary = [item for sublist in poem_df['dic'] for item in sublist]
lyric_dictionary = [item for sublist in lyric_df['dic'] for item in sublist]
dictionary.extend(lyric_dictionary)
for i in dictionary:
    word_freq[i] = word_freq.get(i, 0) + 1

word_freq = {key:val for key, val in word_freq.items() if val >= 3}
    
with open('./lyric_dict.txt', 'w', encoding='utf-8') as f:
    for k, v in word_freq.items():
        f.write(f'{k} {v * 100}\n')

jieba.set_dictionary('./lyric_dict.txt')

In [21]:
def lyric_second_segment(lines):
    words = []
    for line in lines:
        if len(line) == 5:
            words.append(line[0:2])
            words.append(addition_segment(line[2:]))
        if len(line) == 7:
            words.append(line[0:2])
            words.append(line[2:4])
            words.append(addition_segment(line[4:]))
        else:
            tmp = jieba.lcut(line, cut_all=False)
            words.extend(list(filter(lambda x: len(x) >= 2, tmp)))
    return words

In [22]:
lyric_df['dic_total'] = lyric_df['lines'].apply(lyric_second_segment)

Building prefix dict from C:\Users\JUNHONG\history-digital\lyric_dict.txt ...
Dumping model to file cache C:\Users\JUNHONG\AppData\Local\Temp\jieba.uaf15fae91440152dd276ee8f618272a6.cache
Loading model cost 0.238 seconds.
Prefix dict has been built successfully.


In [23]:
run_test_1()

Poem #1
秦川/雄/帝宅/，/函谷/壯/皇居/。
綺/殿/千尋/起/，/離宮/百雉/餘/。
連甍/遙接漢/，/飛觀/迥/凌虛/。
雲日/隱層/闕/，/風煙/出/綺疎/。
Poem #2
遠山/媚/平楚/，/宿雨/漲/清溪/。
沿泝/任/舟檝/，/歡言/無/町畦/。
夜深/忽梦/少年/事/，/梦/啼妆/泪红/阑干
Poem #3
剑外/忽/传收/蓟北/，/初闻/涕泪/满/衣裳/。
却看/妻子/愁/何在/，/漫卷/诗书/喜欲狂/。
白日/放歌/须/纵酒/，/青春/作伴/好/还/乡/。
即/从/巴峡/穿/巫峡/，/便下/襄阳/向/洛阳/。
Poem #4
明月/几时/有/，/把酒/问/青天/。
不知/天上/宫阙/，/今夕/是何/年/。
我欲/乘风/归去/，/又恐/琼楼/玉宇/，/高处/不胜/寒/。
起舞/弄/清影/，/何似/在人/间/。
转/朱阁/，/低绮户/，/照/无眠/。
不应/有恨/，/何事/长向/别时/圆/。
人有/悲欢/离合/，/月有/阴晴/圆缺/，/此事/古难全/。
但愿/人长/久/，/千里/共/婵娟/。


最后一次分词的结果。“却看/妻子/愁/何在”终于分对了，虽然“梦/啼妆/泪红/阑干”分错，但是我们只关心对后三个字的分词，所以无伤大雅。最关键的是，“转/朱阁”终于分出来了，“千里/共/婵娟”的“婵娟”也分出来了。但是上述几个分词词典都没正确地分出“但愿/人长/久”，我看字典里没有“长久”和“人长”，所以我认为是它随便分了。

总结一下我的想法：

假设：五字句AABBB的前两个字AA，七字句AABBCCC的前四个字AA和BB，统一视为可以直接提取出来的“好意象”。当然这个规律也有例外，但是我只找到白居易的“家在虾蟆陵下住”这一个反例，我觉得只是因为白居易本来就专门写通俗化口语化的诗。后三个字的分法大相径庭，所以需要借助分词工具。

做法：
1. 先用唐诗和宋诗的“好意象”构建词典，然后用得到的词典对五字句AABBB的后三个字BBB，七字句AABBCCC的后三个字CCC分词。用对BBB和CCC的分词结果扩充词典。
2. 唐诗的分词完成。分词结果就是好意象+后三个字的分词。
3. 用宋词里的五字句的和七字句的“好意象”扩充词典，对宋词其他句式进行分词，
4. 宋词的分词完成。分词结果就是：
        对于五字句和七字句：好意象+后三个字的分词；
        对于其他句式：整句的分词结果

In [24]:
song_df = lyric_df

poems = []
file_list = os.listdir(POEM_DIR)
for file in file_list:
    if file.startswith('poet.tang'):
        poems.append(pd.read_json(f'{POEM_DIR}{file}'))
tang_df = pd.concat(poems, ignore_index=True)
tang_df['lines'] = tang_df['paragraphs'].apply(cut_lines)
jieba.set_dictionary('./poem_dict.txt')
tang_df['dic_total'] = tang_df['lines'].apply(second_segment)
jieba.set_dictionary('./lyric_dict.txt')

Building prefix dict from C:\Users\JUNHONG\history-digital\poem_dict.txt ...
Loading model from cache C:\Users\JUNHONG\AppData\Local\Temp\jieba.u8c6200af4b343395bea15d660881757b.cache
Loading model cost 0.161 seconds.
Prefix dict has been built successfully.


总之，目前可以使用tang_df['dic_total']和song_df['dic_total']来看唐诗和宋词的分词结果：

In [25]:
#tang_df

In [26]:
#song_df

In [27]:
libai = tang_df[tang_df['author']== '李白']
li_seg = libai['dic_total'].tolist()

In [28]:
#libai

In [29]:
len(libai['paragraphs'])

1207

In [30]:
li_seg[0]

['三十',
 '離宮',
 '樓臺',
 '與天',
 '閣道',
 '步行',
 '美人',
 '愁煙',
 '恩疎',
 '不及',
 '桃李',
 '傷春',
 '淫樂',
 '意何極',
 '金輿',
 '回中',
 '萬乘',
 '黃道',
 '千旗',
 '彩虹',
 '前軍',
 '細柳',
 '後騎',
 '甘泉',
 '豈問',
 '渭川',
 '寧邀',
 '野童',
 '秋暮',
 '瑤池',
 '歸來',
 '未窮']

In [31]:
import operator
from functools import reduce

li_list = reduce(operator.add, li_seg)
#li_list

In [32]:
from collections import Counter

result_li = Counter(li_list)

In [33]:
#result_li

In [35]:
def tang_author_topK(author,K):
    author_seg = tang_df[tang_df['author']==author]['dic_total'].tolist()
    if len(author_seg)==0:return []
    author_list = reduce(operator.add, author_seg) 
    result_author = Counter(author_list)
    return  result_author.most_common(K)

In [37]:
def song_author_topK(author,K):
    author_seg = song_df[song_df['author']==author]['dic_total'].tolist()
    if len(author_seg)==0:return []
    author_list = reduce(operator.add, author_seg) 
    result_author = Counter(author_list)
    return  result_author.most_common(K)

In [41]:
from cnsenti import Sentiment
from zhconv import convert

In [42]:
senti = Sentiment()
test_text= convert('君不見黃河之水天上來，奔流到海不復回。, 君不見高堂明鏡悲白髮，朝如青絲暮成雪。', 'zh-hans')
result = senti.sentiment_count(test_text)
print(result)

Building prefix dict from C:\Users\JUNHONG\history-digital\lyric_dict.txt ...
Loading model from cache C:\Users\JUNHONG\AppData\Local\Temp\jieba.uaf15fae91440152dd276ee8f618272a6.cache
Loading model cost 0.168 seconds.
Prefix dict has been built successfully.


{'words': 26, 'sentences': 2, 'pos': 1, 'neg': 0}


In [43]:
from zhconv import convert

In [48]:
city = ['长安','成都','扬州','杭州','苏州','金陵','幽州','明州','洛阳','钱塘','益州','临安','汴州','汴梁','锦官城','锦城','楼兰','渭城','玉门关']

In [49]:
#reduce(operator.add,libai['paragraphs'].tolist())

In [50]:
def tang_city_count(author):
    result = {}
    for c in city:
        count = 0
        author_l = tang_df[tang_df['author']== author]
        if len( author_l)==0:
            return {}
        l = reduce(operator.add, author_l['paragraphs'].tolist())
        for j in l:
            if c in convert(j, 'zh-hans'):
                count = count + 1
        result[c] = count
    return result
            

In [51]:
def song_city_count(author):
    result = {}
    for c in city:
        count = 0
        author_l = song_df[song_df['author']== author]
        if len( author_l)==0:
            return {}
        l = reduce(operator.add, author_l['paragraphs'].tolist())
        for j in l:
            if c in convert(j, 'zh-hans'):
                count = count + 1
        result[c] = count
    return result

In [53]:
tang_female = ['薛涛','刘采春','鱼玄机','李冶']

In [55]:
for i in tang_female:
    print(i,tang_city_count(convert(i, 'zh-hant')))

薛涛 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 1, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 1, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
刘采春 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
鱼玄机 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
李冶 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}


In [56]:
for i in tang_female:
    print(i,tang_author_topK(convert(i, 'zh-hant'),10))

薛涛 [('不得', 13), ('芙蓉', 6), ('花落', 4), ('同心', 4), ('人間', 4), ('夕陽', 4), ('四五', 4), ('春風', 3), ('今日', 3), ('萬里', 3)]
刘采春 [('桐廬', 2), ('昨日', 2), ('不喜', 1), ('秦淮', 1), ('生憎', 1), ('江上', 1), ('載兒', 1), ('夫壻', 1), ('經歲', 1), ('又經', 1)]
鱼玄机 [('何處', 6), ('鸚鵡', 4), ('多情', 4), ('何事', 3), ('滿庭', 3), ('鴛鴦', 3), ('朱弦', 3), ('楊柳', 3), ('客舟', 2), ('風雨', 2)]
李冶 [('相思', 6), ('相逢', 3), ('一時', 3), ('寂寞', 2), ('別後', 2), ('巫山', 2), ('流泉', 2), ('一彈', 2), ('海水', 2), ('暮雨', 2)]


In [57]:
song_female = ['李清照','张玉娘','吴淑姬','朱淑真']

In [58]:
for i in song_female:
    print(i,song_city_count(convert(i, 'zh-hant')))

李清照 {'长安': 1, '成都': 0, '扬州': 1, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
张玉娘 {}
吴淑姬 {}
朱淑真 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}


In [59]:
for i in song_female:
    print(i,song_author_topK(convert(i, 'zh-hant'),10))

李清照 [('多少', 6), ('萧萧', 6), ('憔悴', 6), ('瑞脑', 6), ('情怀', 5), ('香消', 5), ('梅花', 5), ('旧时', 5), ('庭院', 5), ('如今', 5)]
张玉娘 []
吴淑姬 []
朱淑真 [('寂寞', 6), ('何处', 6), ('东风', 5), ('阑干', 5), ('芳草', 5), ('不忍', 4), ('梨花', 4), ('起来', 4), ('一枝', 4), ('恼人', 3)]


In [62]:
li_friend = ['李白','杜甫','白居易','劉禹錫']
han_friend = ['韓愈','張籍','孟郊','盧殷']
liu_friend = ['劉長卿','劉方平','李嘉祐','皇甫冉']
wei_friend = ['杜牧','李商隱','韋莊','許渾']
su_friend = ['苏轼','王安石','韩琦','欧阳修','黄庭坚']

In [63]:
for i in li_friend:
    print(i,tang_city_count(i))

李白 {'长安': 35, '成都': 1, '扬州': 6, '杭州': 0, '苏州': 0, '金陵': 30, '幽州': 7, '明州': 0, '洛阳': 26, '钱塘': 0, '益州': 0, '临安': 1, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 6, '楼兰': 5, '渭城': 1, '玉门关': 2}
杜甫 {'长安': 26, '成都': 10, '扬州': 3, '杭州': 1, '苏州': 0, '金陵': 0, '幽州': 2, '明州': 0, '洛阳': 12, '钱塘': 0, '益州': 2, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 6, '锦城': 7, '楼兰': 2, '渭城': 0, '玉门关': 0}
白居易 {'长安': 51, '成都': 2, '扬州': 4, '杭州': 17, '苏州': 19, '金陵': 0, '幽州': 0, '明州': 1, '洛阳': 50, '钱塘': 3, '益州': 0, '临安': 1, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 1, '楼兰': 0, '渭城': 2, '玉门关': 0}
劉禹錫 {'长安': 19, '成都': 3, '扬州': 7, '杭州': 0, '苏州': 4, '金陵': 1, '幽州': 0, '明州': 1, '洛阳': 27, '钱塘': 2, '益州': 1, '临安': 0, '汴州': 1, '汴梁': 0, '锦官城': 0, '锦城': 1, '楼兰': 0, '渭城': 1, '玉门关': 0}


In [64]:
for i in han_friend:
    print(i,tang_city_count(i))

韓愈 {'长安': 9, '成都': 0, '扬州': 1, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 1, '明州': 0, '洛阳': 6, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 2, '汴梁': 0, '锦官城': 0, '锦城': 1, '楼兰': 0, '渭城': 0, '玉门关': 0}
張籍 {'长安': 14, '成都': 0, '扬州': 2, '杭州': 0, '苏州': 0, '金陵': 4, '幽州': 0, '明州': 0, '洛阳': 9, '钱塘': 0, '益州': 1, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 1, '楼兰': 0, '渭城': 0, '玉门关': 0}
孟郊 {'长安': 18, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 4, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 1, '渭城': 0, '玉门关': 0}
盧殷 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}


In [65]:
for i in liu_friend:
    print(i,tang_city_count(i))

劉長卿 {'长安': 11, '成都': 0, '扬州': 3, '杭州': 0, '苏州': 0, '金陵': 8, '幽州': 1, '明州': 0, '洛阳': 9, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
劉方平 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 3, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
李嘉祐 {'长安': 1, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 2, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
皇甫冉 {'长安': 4, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 2, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}


In [66]:
for i in wei_friend:
    print(i,tang_city_count(i))

杜牧 {'长安': 5, '成都': 0, '扬州': 7, '杭州': 0, '苏州': 0, '金陵': 3, '幽州': 0, '明州': 0, '洛阳': 3, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
李商隱 {'长安': 1, '成都': 2, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 2, '幽州': 0, '明州': 0, '洛阳': 3, '钱塘': 0, '益州': 1, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 1, '楼兰': 0, '渭城': 0, '玉门关': 0}
韋莊 {'长安': 11, '成都': 0, '扬州': 1, '杭州': 0, '苏州': 0, '金陵': 2, '幽州': 0, '明州': 0, '洛阳': 4, '钱塘': 1, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 1, '楼兰': 1, '渭城': 1, '玉门关': 0}
許渾 {'长安': 8, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 1, '幽州': 0, '明州': 0, '洛阳': 4, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}


In [67]:
for i in su_friend:
    print(i,song_city_count(i))

苏轼 {'长安': 7, '成都': 3, '扬州': 3, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 1, '钱塘': 5, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
王安石 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
韩琦 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
欧阳修 {'长安': 3, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 3, '钱塘': 1, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
黄庭坚 {'长安': 0, '成都': 0, '扬州': 2, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 1, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}


In [68]:
for i in su_friend:
    print(i,song_author_topK(i,10))

苏轼 [('归去', 44), ('何处', 39), ('使君', 28), ('佳人', 26), ('人间', 23), ('明月', 22), ('江南', 20), ('尊前', 20), ('千里', 19), ('风流', 18)]
王安石 [('隔岸', 5), ('今日', 4), ('是何', 4), ('不肯', 4), ('世间', 4), ('相逢', 4), ('故人', 3), ('春风', 3), ('茫然', 3), ('弹指', 3)]
韩琦 [('滴尽', 2), ('胭脂', 2), ('谁向', 2), ('花前', 2), ('人远', 2), ('空翠', 2), ('安阳', 2), ('形势', 2), ('西州', 2), ('和气', 2)]
欧阳修 [('相思', 26), ('东风', 15), ('把酒', 14), ('红粉', 14), ('西湖', 13), ('今日', 13), ('阑干', 12), ('尊前', 12), ('风流', 12), ('无人', 11)]
黄庭坚 [('万里', 19), ('何处', 15), ('归来', 14), ('尊前', 12), ('风流', 12), ('明月', 11), ('不得', 10), ('桃李', 10), ('年少', 9), ('江南', 9)]


In [69]:
for i in li_friend:
    print(i,tang_author_topK(i,10))

李白 [('不可', 75), ('黃金', 68), ('春風', 65), ('萬里', 60), ('白日', 57), ('明月', 52), ('青天', 49), ('相思', 49), ('天地', 47), ('青雲', 44)]
杜甫 [('萬里', 75), ('乾坤', 45), ('風塵', 44), ('不可', 41), ('白帝', 41), ('故人', 40), ('回首', 38), ('巫峽', 36), ('干戈', 35), ('白頭', 35)]
白居易 [('不知', 107), ('何處', 104), ('今日', 104), ('人間', 93), ('不如', 91), ('不得', 88), ('何人', 84), ('平生', 72), ('三十', 71), ('今朝', 69)]
劉禹錫 [('何處', 45), ('今日', 30), ('從此', 29), ('人間', 28), ('洛陽', 27), ('故人', 23), ('春深', 22), ('如今', 21), ('深春', 20), ('旌旗', 20)]
